<a href="https://colab.research.google.com/github/tayfununal/2-Hidden-Layer-Neural-Networks/blob/master/ono_hidden_layer_nn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install playground-data

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import plygdata as pg

In [ ]:
data = pg.dataset.DataGenerator.classify_xor(1000,noise=0.1)

In [ ]:
df = pd.DataFrame(data).head()
df

In [ ]:
#Seperate the data as train set and test set with 0.2
X_train, y_train, X_test, y_test = pg.split_data(data, validation_size=0.2)
y_train.shape

In [ ]:
#Change the value of -1 in y_train and y_test with 0
y_train = np.array([0 if y_train[i,0]==-1 else 1 for i in range(len(y_train))])
y_train = y_train.reshape(800,1)
y_test = np.array([0 if y_test[i,0]==-1 else 1 for i in range(len(y_test))])
y_test = y_test.reshape(200,1)

In [ ]:
X_train = X_train.T
y_train = y_train.T
X_test = X_test.T
y_test = y_test.T

In [ ]:
print("Shape of X_train:", X_train.shape,
      "\nShape of y_train:", y_train.shape,
      "\nShape of X_test:", X_test.shape,
      "\nShape of y_test:", y_test.shape)

In [ ]:
plt.figure(figsize=(8,8))
for i in range(X_train.shape[1]):
  if y_train[0,i]==1:
    plt.scatter(X_train[0,i], X_train[1,i] , c='#0877bd' , alpha=0.5)
  else:
    plt.scatter(X_train[0,i], X_train[1,i] , c='#f59322' , alpha=0.5)

for i in range(X_test.shape[1]):
  if y_test[0,i]==1:
    plt.scatter(X_test[0,i], X_test[1,i] , c='#0877bd', edgecolors='#e8eaeb', s=80)
  else:
    plt.scatter(X_test[0,i], X_test[1,i] , c='#f59322', edgecolors='#e8eaeb' , s=80)

In [ ]:
def initialization_parameters(x, y, num_node):
  W1 = np.random.randn(num_node, x.shape[0]) * 0.001
  b1 = np.zeros((num_node,1))

  W2 = np.random.randn(y.shape[0],num_node) * 0.001
  b2 = np.zeros((y.shape[0],1))

  assert W1.shape == (num_node,x.shape[0])
  assert b1.shape == (num_node, 1)

  assert W2.shape == (y.shape[0], num_node)
  assert b2.shape == (y.shape[0], 1)

  parameters = {'W1':W1,
                'b1':b1,
                'W2':W2,
                'b2':b2}
  return parameters

def sigmoid(x):
  return 1/(1+np.exp(-x))

def relu(x):
  return np.maximum(0,x)

def reluDerivative(x):
  x[x<=0] = 0
  x[x>0] = 1
  return x

def forward_prop(x,parameters):
  W1 = parameters['W1']
  b1 = parameters['b1']
  W2 = parameters['W2']
  b2 = parameters['b2']
  
  Z1 = np.dot(W1, x) + b1
  A1 = relu(Z1)
  Z2 = np.dot(W2, A1) + b2
  A2 = sigmoid(Z2)

  assert (A2.shape == (1, x.shape[1]))
  cache = {
      'Z1' : Z1,
      'A1' : A1,
      'Z2' : Z2,
      'A2' : A2
  }
  return A2, cache

def backward_prop(x, y, parameters, cache, learning_rate = 0.1):
  m = y.shape[1]
  
  W1 = parameters['W1']
  b1 = parameters['b1']
  W2 = parameters['W2']
  b2 = parameters['b2']

  A1 = cache['A1']
  A2 = cache['A2']
  Z1 = cache['Z1']
  
  # Calculations of backward propagation: dW1, db1, dW2, db2
  dZ2 = A2 - y
  dW2 = (1 / m) * np.dot(dZ2, A1.T)
  db2 = (1 / m) * np.sum(dZ2, axis = 1, keepdims = True)

  dZ1 = np.multiply(np.dot(W2.T, dZ2), reluDerivative(Z1))
  dW1 = (1 / m) * np.dot(dZ1, x.T)
  db1 = (1 / m) * np.sum(dZ1 , axis = 1, keepdims = True)

  # Updating parameters
  W1 = W1 - learning_rate * dW1
  b1 = b1 - learning_rate * db1
  W2 = W2 - learning_rate * dW2
  b2 = b2 - learning_rate * db2
  
  parameters = {'W1':W1,
                'b1':b1,
                'W2':W2,
                'b2':b2}
  return parameters

def cross_entropy_cost(y, A2):
  m = y.shape[1]
  
  cross_entropy = np.multiply(np.log(A2 + 1e-15), y) + np.multiply((1 - y), np.log(1 - A2 + 1e-15))
  cost = - 1.0 / m * np.sum(cross_entropy)

  # Squeezing to avoid unnecessary dimensions 
  cost = np.squeeze(cost) 
  return cost  

In [ ]:
def nn_model(x, y, parameters, number_of_iter = 1000):
  cost_value = {}
  A2, cache = forward_prop(x, parameters)
  cost = cross_entropy_cost(y, A2)
  cost_value[1] = cost

  for i in range(1,number_of_iter):
    parameters = backward_prop(x, y, parameters, cache, learning_rate=0.1)
    A2, cache = forward_prop(x, parameters)
    cost = cross_entropy_cost(y, A2)
    
    #cost_value[i+1] = cost
    
    if (i+1) % 100 == 0:
     cost_value[i+1] = cost
     print("%i."%(i+1),cost)
  return cost_value, parameters

In [ ]:
#np.random.seed(0)
parameters = initialization_parameters(X_test, y_test, 20)
cost_value, parameters = nn_model(X_train, y_train, parameters, number_of_iter = 1000)

In [ ]:
print(cost_value)

In [ ]:
clor= [pg.DataColor.get_rgb(i) for i in range(-1,2)]
clor